   #### Mawrer Amed Ramirez Martinez ms723737@iteso.mx
   #### Avance de TOG IDI 4

#  Ajuste de Dataset (Preparacion)


In [9]:
from datetime import datetime
import pandas as pd
import os, glob

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
SOURCE_TYPE = "EURUSD_ASK"
FRECUENCIA = "1H"
NUM_FEATURES = 0


index = None
if FRECUENCIA == "1D":
    index = 1
elif FRECUENCIA == "1H":
    index = 2
elif FRECUENCIA == "1M":
    index = 3
elif FRECUENCIA == "2H":
    index = 4
elif FRECUENCIA == "4H":
    index = 5

In [34]:
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Datasets/Junio2020/dukas/EURUSD/'+FRECUENCIA+'/')
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

EURUSD_Candlestick_1_Hour_ASK_31.12.2003-30.05.2020.csv
EURUSD_Candlestick_1_Hour_BID_31.12.2003-30.05.2020.csv


## Read CSV

### Read Ask Dataset

In [37]:
index_ask = 0
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y %H:%M:%S.%f')
DATASET_ASK = pd.read_csv(filenames[index_ask], header=0, parse_dates=['Gmt time'], date_parser=dateparse)
DATASET_ASK.head()

,Gmt time,Open,High,Low,Close,Volume
0,2004-01-01 00:00:00,1.25950,1.25960,1.25826,1.25910,6654.2002
1,2004-01-01 01:00:00,1.25958,1.25960,1.25947,1.25951,660.6000
2,2004-01-01 02:00:00,1.25955,1.25962,1.25944,1.25951,2091.0000
3,2004-01-01 03:00:00,1.25950,1.25965,1.25834,1.25869,10003.0996
4,2004-01-01 04:00:00,1.25908,1.25960,1.25843,1.25950,7603.3999


### Read Bid Dataset

In [41]:
index_bid = 1
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y %H:%M:%S.%f')
DATASET_BID = pd.read_csv(filenames[index_bid], header=0, parse_dates=['Gmt time'], date_parser=dateparse, 
                          names=['Gmt time','Open_Bid','High_Bid',"Low_Bid","Close_Bid","Volume_Bid"])
DATASET_BID.head()

,Gmt time,Open_Bid,High_Bid,Low_Bid,Close_Bid,Volume_Bid
0,2004-01-01 00:00:00,1.25934,1.25948,1.25814,1.25900,7823.1001
1,2004-01-01 01:00:00,1.25942,1.25948,1.25934,1.25941,461.4000
2,2004-01-01 02:00:00,1.25945,1.25948,1.25927,1.25936,1839.8000
3,2004-01-01 03:00:00,1.25934,1.25953,1.25821,1.25854,13349.2998
4,2004-01-01 04:00:00,1.25898,1.25948,1.25829,1.25940,8422.5000


### Add Col 'day_name' and 'weekday'

In [14]:
DF_EURUSD_ASK['day_name'] = DF_EURUSD_ASK['Gmt time'].dt.day_name()
DF_EURUSD_ASK['weekday'] = DF_EURUSD_ASK['Gmt time'].dt.weekday
print(DF_EURUSD_ASK.head(5))

             Gmt time     Open     High      Low    Close     Volume day_name  \
0 2018-01-01 22:00:00  1.20143  1.20173  1.20062  1.20158  1623.5100   Monday   
1 2018-01-01 23:00:00  1.20158  1.20161  1.20044  1.20110  6297.2402   Monday   
2 2018-01-02 00:00:00  1.20109  1.20178  1.20096  1.20162  3564.4600  Tuesday   
3 2018-01-02 01:00:00  1.20162  1.20232  1.20141  1.20159  6219.6899  Tuesday   
4 2018-01-02 02:00:00  1.20157  1.20165  1.20081  1.20096  5386.5098  Tuesday   

   weekday  
0        0  
1        0  
2        1  
3        1  
4        1  


### Add Col 'pct_range_Close' 

In [15]:
DF_EURUSD_ASK['Pct_Range_Close'] = DF_EURUSD_ASK['Close'].pct_change()
DF_EURUSD_ASK['Pct_Range_Close'].fillna(0, inplace=True)
print(DF_EURUSD_ASK.head())
print(DF_EURUSD_ASK.shape)


             Gmt time     Open     High      Low    Close     Volume day_name  \
0 2018-01-01 22:00:00  1.20143  1.20173  1.20062  1.20158  1623.5100   Monday   
1 2018-01-01 23:00:00  1.20158  1.20161  1.20044  1.20110  6297.2402   Monday   
2 2018-01-02 00:00:00  1.20109  1.20178  1.20096  1.20162  3564.4600  Tuesday   
3 2018-01-02 01:00:00  1.20162  1.20232  1.20141  1.20159  6219.6899  Tuesday   
4 2018-01-02 02:00:00  1.20157  1.20165  1.20081  1.20096  5386.5098  Tuesday   

   weekday  Pct_Range_Close  
0        0         0.000000  
1        0        -0.000399  
2        1         0.000433  
3        1        -0.000025  
4        1        -0.000524  
(15023, 9)


### Differnce Function

In [16]:
def difference(dataset, interval, col_name):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.DataFrame(diff, columns = [col_name + "_diff"])

### Add Col 'Pct_Range_Close_diff' 

In [17]:
DF_EURUSD_ASK['Pct_Range_Close_diff'] = difference(DF_EURUSD_ASK['Pct_Range_Close'],1,'Pct_Range_Close').shift(1)
DF_EURUSD_ASK['Pct_Range_Close_diff'].fillna(0, inplace=True)
print(DF_EURUSD_ASK.head())
print(DF_EURUSD_ASK.shape)

             Gmt time     Open     High      Low    Close     Volume day_name  \
0 2018-01-01 22:00:00  1.20143  1.20173  1.20062  1.20158  1623.5100   Monday   
1 2018-01-01 23:00:00  1.20158  1.20161  1.20044  1.20110  6297.2402   Monday   
2 2018-01-02 00:00:00  1.20109  1.20178  1.20096  1.20162  3564.4600  Tuesday   
3 2018-01-02 01:00:00  1.20162  1.20232  1.20141  1.20159  6219.6899  Tuesday   
4 2018-01-02 02:00:00  1.20157  1.20165  1.20081  1.20096  5386.5098  Tuesday   

   weekday  Pct_Range_Close  Pct_Range_Close_diff  
0        0         0.000000              0.000000  
1        0        -0.000399             -0.000399  
2        1         0.000433              0.000832  
3        1        -0.000025             -0.000458  
4        1        -0.000524             -0.000499  
(15023, 10)


### Moving Average (MA) Function

In [18]:
#This function add a new column with the MA calculed, with value n
#col_name is the value to be used as base.
def MA(dataset, n, col_name, new_col): 
    MA = pd.Series(dataset[col_name].rolling(n).mean(), name = 'MA_' + col_name +'_'+ str(n)) 
    dataset = dataset.join(MA)  
    return dataset


### Add Col 'MA_Close_5' 

In [19]:
DF_EURUSD_ASK = MA(DF_EURUSD_ASK,5,'Close','MA_Close_5')
DF_EURUSD_ASK['MA_Close_5'].fillna(0, inplace=True)
DF_EURUSD_ASK.head(5)

,Gmt time,Open,High,Low,Close,Volume,day_name,weekday,Pct_Range_Close,Pct_Range_Close_diff,MA_Close_5
0,2018-01-01 22:00:00,1.20143,1.20173,1.20062,1.20158,1623.5100,Monday,0,0.000000,0.000000,0.00000
1,2018-01-01 23:00:00,1.20158,1.20161,1.20044,1.20110,6297.2402,Monday,0,-0.000399,-0.000399,0.00000
2,2018-01-02 00:00:00,1.20109,1.20178,1.20096,1.20162,3564.4600,Tuesday,1,0.000433,0.000832,0.00000
3,2018-01-02 01:00:00,1.20162,1.20232,1.20141,1.20159,6219.6899,Tuesday,1,-0.000025,-0.000458,0.00000
4,2018-01-02 02:00:00,1.20157,1.20165,1.20081,1.20096,5386.5098,Tuesday,1,-0.000524,-0.000499,1.20137


### Add Col 'MA_Close_25' 

In [20]:
DF_EURUSD_ASK = MA(DF_EURUSD_ASK,25,'Close','MA_Close_25')
DF_EURUSD_ASK['MA_Close_25'].fillna(0, inplace=True)
DF_EURUSD_ASK[22:27]

,Gmt time,Open,High,Low,Close,Volume,day_name,weekday,Pct_Range_Close,Pct_Range_Close_diff,MA_Close_5,MA_Close_25
22,2018-01-02 20:00:00,1.20539,1.20609,1.20520,1.20593,6834.2998,Tuesday,1,0.000473,-0.000266,1.205036,0.000000
23,2018-01-02 21:00:00,1.20593,1.20654,1.20549,1.20590,5248.0298,Tuesday,1,-0.000025,-0.000498,1.205230,0.000000
24,2018-01-02 22:00:00,1.20590,1.20594,1.20558,1.20578,3081.7400,Tuesday,1,-0.000100,-0.000075,1.205488,1.204019
25,2018-01-02 23:00:00,1.20577,1.20664,1.20573,1.20618,3717.2200,Tuesday,1,0.000332,0.000431,1.205830,1.204203
26,2018-01-03 00:00:00,1.20619,1.20641,1.20496,1.20595,5899.1499,Wednesday,2,-0.000191,-0.000522,1.205948,1.204397


### Add Col 'MA_Close_50' 

In [21]:
DF_EURUSD_ASK = MA(DF_EURUSD_ASK,50,'Close','MA_Close_50')
DF_EURUSD_ASK['MA_Close_50'].fillna(0, inplace=True)
DF_EURUSD_ASK[48:52]

,Gmt time,Open,High,Low,Close,Volume,day_name,weekday,Pct_Range_Close,Pct_Range_Close_diff,MA_Close_5,MA_Close_25,MA_Close_50
48,2018-01-03 22:00:00,1.20146,1.20170,1.20110,1.20115,3023.8899,Wednesday,2,-0.000258,-0.000408,1.201676,1.203480,0.000000
49,2018-01-03 23:00:00,1.20114,1.20143,1.20069,1.20112,4165.0698,Wednesday,2,-0.000025,0.000233,1.201332,1.203293,1.203656
50,2018-01-04 00:00:00,1.20112,1.20115,1.20046,1.20102,5685.7998,Thursday,3,-0.000083,-0.000058,1.201206,1.203087,1.203645
51,2018-01-04 01:00:00,1.20101,1.20112,1.20052,1.20080,4884.4199,Thursday,3,-0.000183,-0.000100,1.201110,1.202881,1.203639


### Add Col 'MA_Close_200' 

In [22]:
DF_EURUSD_ASK = MA(DF_EURUSD_ASK,200,'Close','MA_Close_200')
DF_EURUSD_ASK['MA_Close_200'].fillna(0, inplace=True)
DF_EURUSD_ASK[198:202]

,Gmt time,Open,High,Low,Close,Volume,day_name,weekday,Pct_Range_Close,Pct_Range_Close_diff,MA_Close_5,MA_Close_25,MA_Close_50,MA_Close_200
198,2018-01-12 04:00:00,1.20516,1.20549,1.20459,1.20538,5759.1401,Friday,4,0.000174,-0.000208,1.205134,1.200968,1.198495,0.000000
199,2018-01-12 05:00:00,1.20538,1.20574,1.20402,1.20443,5914.2402,Friday,4,-0.000788,-0.000962,1.204850,1.201357,1.198699,1.200896
200,2018-01-12 06:00:00,1.20443,1.20524,1.20426,1.20485,6369.0498,Friday,4,0.000349,0.001137,1.204908,1.201740,1.198924,1.200912
201,2018-01-12 07:00:00,1.20484,1.20756,1.20458,1.20627,14090.5400,Friday,4,0.001179,0.000830,1.205220,1.202222,1.199193,1.200938


### Save CSV File 'DF_EURUSD_ASK_1H_PROC.csv'

In [30]:
#Get the Final number of Features
NUM_FEATURES = str(DF_EURUSD_ASK.shape[1])
filename = SOURCE_TYPE + '_' + FRECUENCIA + '_' + NUM_FEATURES + 'F.csv'
print(filename)

EURUSD_ASK_1H_14F.csv


In [31]:
pwd = os.getcwd()
filename_path = os.path.join(pwd, r'../../../Datasets/Junio2020/procesados/' + filename)
DF_EURUSD_ASK.to_csv(filename_path, header=True, index=False)